In [2]:
import pandas as pd
import os
import urllib.parse
import pdfkit
import re

# === CONFIG ===
input_csv = "CallingList.csv"   # Replace with your actual CSV file name
output_folder = "."      # Folder to save the HTML files
# form_url_base = (
#     "https://docs.google.com/forms/d/e/1FAIpQLSddgJb-7qSWjw4vIjB-6rAddcIrCSTreRmbnNNAst2I_bd8dA/viewform?usp=pp_url"
# )
#https://forms.gle/4NozGynTSdfJSa1h7

#https://docs.google.com/forms/d/e/1FAIpQLSeLBvPv6FMRGs6Lt9F4iW7HLdXfzDe3bh55x5o5fyooKUA20g/viewform?usp=pp_url&entry.1320622647=Confirmed+Presence&entry.1907793231=Ratish+Moondra&entry.1678180232=Ratish+Moondra&entry.629158787=8108711149

form_url_base = (
    "https://docs.google.com/forms/d/e/1FAIpQLSeLBvPv6FMRGs6Lt9F4iW7HLdXfzDe3bh55x5o5fyooKUA20g/viewform?usp=pp_url"
)


# === CREATE OUTPUT FOLDER IF NOT EXISTS ===
os.makedirs(output_folder, exist_ok=True)

# === READ CSV ===
df = pd.read_csv(input_csv)

df.head()

# Plan B: New Google Form Link created for Event Registration : https://forms.gle/Asvp6qqAv5utMb1x8
# Marksheet Submission Link : https://forms.gle/YgKUfYDDkNWTYJHUA


# Clean column names if needed
df.columns = df.columns.str.strip()

# === PROCESS EACH SAMITI MEMBER ===
for samiti_name, group in df.groupby("Samit Member Name"):
    # Prepare safe filename (replace spaces, dots, commas)
    safe_name = samiti_name.replace(" ", " ").replace(".", "").replace(",", "")
    # for honorific in ["Shri ", "Shri. ", "ji", "Ji", "Shri"]:
    #     clean_safe_name = safe_name.replace(honorific, "")
    # for honorific in ["Shri ", "Shri. ", "ji", "Ji", "Shri"]:
    #     clean_safe_name = safe_name.replace(honorific, "")        
    safe_name = safe_name.strip()

    fName = safe_name.replace(" ", "")
    filename = f"CallingList{fName}.html"
    filepath = os.path.join(output_folder, filename)
    

    # Start HTML content
    html = f"""
    <html>
    <head>
        <meta charset="UTF-8">
        <title>Calling List - {samiti_name}</title>
        <style>
            body {{ font-family: Arial, sans-serif; margin: 20px; }}
            h2 {{ text-align: center; }}
            table {{ width: 100%; border-collapse: collapse; margin-top: 20px; }}
            th, td {{ border: 1px solid #ccc; padding: 8px; text-align: left; }}
            th {{ background-color: #f2f2f2; }}
            a {{ color: #007bff; text-decoration: none; }}
            a:hover {{ text-decoration: underline; }}
        </style>
    </head>
    <body>
        <h2>Calling List - {samiti_name}</h2>
        <div style="font-family: Arial, sans-serif; color: #333; line-height: 1.4; padding: 8px; max-width: 100%; box-sizing: border-box; font-size: 14px;">
            <p style="font-weight: bold; margin-bottom: 6px; font-size: 15px;">
                कृपया निम्नलिखित सरल चरणों का पालन करें:
            </p>
            <ul style="padding-left: 18px; margin: 0; font-size: 8px;">
                <li><strong>Mobile Number</strong> पर क्लिक करें &mdash; कॉल अपने आप डायल हो जाएगी</li>
                <li><strong>SMS</strong> पर क्लिक करें &mdash; आपके मोबाइल से आमंत्रण संदेश जाएगा</li>
                <li><strong>WhatsApp</strong> पर क्लिक करें &mdash; व्हाट्सएप खुलेगा और संदेश अपने आप पोस्ट हो जाएगा</li>
                <li>समाजबंधु से बात करने के बाद <strong>Log Call</strong> पर क्लिक करें ताकि आपकी कॉल का विवरण सीधे Google Form पर सेव हो जाए</li>
                <li>आप समाजबंधु को <strong>Utsav / Mark Sheet Submission / Dinner</strong> की जानकारी दें</li>
            </ul>
        </div>        
        <table>
            <tr>
                <th>#</th>
                <th>Assigned Person Name</th>
                <th>Mobile Number</th>
                <th>Call Log Link</th>
                <th>Call Status</th>
                <th>Registered</th>
                <th>Remarks</th>
            </tr>
    """
    # Sort the group DataFrame by 'Assigned Person' alphabetically
    group_sorted = group.sort_values(by="Assigned Person")
    i = 1
    temp_samiti_name = samiti_name
    if samiti_name.lower().startswith("shri"):
        temp_samiti_name = samiti_name[4:].lstrip()
        # Remove "ji" or "Ji" anywhere in the name (word boundary or attached)
        temp_samiti_name = re.sub(r'ji\b', '', temp_samiti_name, flags=re.IGNORECASE)
    for _, row in group_sorted.iterrows():
        assigned_person = row["Assigned Person"]
        Status = row["Status"]
        Registered = row["Registered"]
        Remarks = row["Remarks"]
        Address = row["Address"]

        # Determine background color based on Status
        row_color = " style='background-color: #ffcccc;'" if str(Status).strip().lower() == "not called" else ""        

        # Custom WhatsApp message

        whatsapp_message = (
            "🙏 Jai Mahesh!\n"
            f"Namskar {assigned_person} ji,\n\n"
            "Welcome to *Malad Utsav 2025* 🎉\n\n"
            "📅 Date: 9 Nov 2025 (Sunday)\n"
            "🕠 Time: 5:30 PM Onwards\n"
            "📍 Venue: Maheshwari Plot, Borivali (W)\n\n"
            "Please confirm your participation using the google form below 👇\n"
            "https://forms.gle/Asvp6qqAv5utMb1x8 \n\n"
            "For Marksheet submission:   👇\n"
            "https://forms.gle/YgKUfYDDkNWTYJHUA \n\n"
            "if you face any issues with marksheet submission, Whatsapp  # 9820454734\n\n"
            f"For any queries, contact Karykarta {temp_samiti_name}.\n\n"
        )
        
        # Encode the message for URL
        encoded_whatsapp_message = urllib.parse.quote_plus(whatsapp_message)
        
        mobile = str(row["Mobile Number"]).strip()

        # Encode for URL
        encoded_person = urllib.parse.quote(assigned_person)
        encoded_mobile = urllib.parse.quote(mobile)
        encoded_samiti = urllib.parse.quote(samiti_name)
        #encoded_address = urllib.parse.quote(Address)

        # Google Form Prefilled Link
        call_log_link = (
            f"{form_url_base}"
            f"&entry.1320622647=Called"
            f"&entry.1907793231={encoded_samiti}"
            f"&entry.629158787={encoded_mobile}"
            f"&entry.1678180232={encoded_person}"
        )

        # Create tel: and WhatsApp links
        tel_link = f"<a onclick=\"this.style.color='green'\" href='tel:+91{mobile}'>+91 {mobile}</a>"
        # SMS link
        sms_link = f'<a href="sms:+91{mobile}?body={encoded_whatsapp_message}" target="_blank" onclick="this.style.color=\'green\'">SMS</a>'

        # WhatsApp link
        whatsapp_link = f'<a href="https://wa.me/91{mobile}?text={encoded_whatsapp_message}" target="_blank" onclick="this.style.color=\'green\'">WhatsApp</a>'


        html += f"""
            <tr {row_color}>
                <td>{i}</td>
                <td>{assigned_person}</td>
                <td>{tel_link}<br/>{sms_link}<br/>{whatsapp_link}<br/>{Address}</td>
                <td><a onclick=\"this.style.color='green'\" href="{call_log_link}" target="_blank">Log Call</a></td>
                <td>{Status}</td>
                <td>{Registered}</td>
                <td>{Remarks}</td>
            </tr>
        """
        i = i+1

    html += """
        </table>
    </body>
    </html>
    """
    # from weasyprint import HTML
    # Save the HTML file
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(html)

    # Convert local HTML file to PDF
    # HTML(filepath).write_pdf(pdf_filepath)

print(f"✅ HTML files successfully created in folder: {output_folder}")


✅ HTML files successfully created in folder: .


In [3]:
# DASHBOARD 
import pandas as pd
import urllib.parse

# === CONFIG ===
input_csv = "CallingList.csv"      # CSV must have 'Samiti Member Phone Number'
output_html = "DashboardSummary.html"

# === READ DATA ===
df = pd.read_csv(input_csv)
df.columns = df.columns.str.strip()

# === NORMALIZE STRINGS ===
for col in ["Status", "Registered", "Samit Member Name"]:
    df[col] = df[col].astype(str).str.strip()

# === CLASSIFY STATUSES ===
called_statuses = [
    "Confirmed Presence",
    "Maybe Attending",
    "Phone Busy",
    "Not Reachable",
    "Not Coming"
]

def is_called(status):
    return status in called_statuses

# === GROUP AND SUMMARIZE ===
summary = (
    df.groupby(["Samit Member Name", "Samiti Member Phone Number"])
      .agg(
          Total_Assigned=("Assigned Person", "count"),
          Called_Count=("Status", lambda x: sum(is_called(s) for s in x)),
          Confirmed_Presence=("Status", lambda x: sum(str(s).strip().lower() == "confirmed presence" for s in x)),
          Maybe_Attending=("Status", lambda x: sum(str(s).strip().lower() == "maybe attending" for s in x)),
          Phone_Busy=("Status", lambda x: sum(str(s).strip().lower() == "phone busy" for s in x)),
          Not_Reachable=("Status", lambda x: sum(str(s).strip().lower() == "not reachable" for s in x)),
          Not_Coming=("Status", lambda x: sum(str(s).strip().lower() == "not coming" for s in x)),
          Registered=("Registered", lambda x: sum(str(s).strip().lower() == "registered" for s in x)),
      )
      .reset_index()
)

# Calculate totals for summary
total_assigned = summary['Total_Assigned'].sum()
total_called = summary['Called_Count'].sum()
call_percentage = (total_called / total_assigned * 100) if total_assigned > 0 else 0

# 🔽 Sort by Called_Count descending
summary = summary.sort_values(by="Samit Member Name", ascending=True)

# === GENERATE SIMPLE DASHBOARD HTML ===
# Start of HTML
html_start = """
<html>
<head>
<meta charset="UTF-8">
<title>Calling Progress Dashboard</title>
<style>
    body {
        font-family: Arial, sans-serif;
        margin: 20px;
        color: #333;
        background-color: #fff;
    }
    h2 {
        text-align: center;
        margin-bottom: 20px;
    }
    .summary-box {
        background: #f8f9fa;
        border: 1px solid #dee2e6;
        border-radius: 8px;
        padding: 15px;
        margin: 20px auto;
        max-width: 600px;
        text-align: center;
    }
    .summary-box p {
        margin: 5px 0;
        font-size: 16px;
    }
    .summary-box .highlight {
        font-weight: bold;
        color: #007bff;
    }
    table {
        width: 100%;
        border-collapse: collapse;
        font-size: 14px;
        margin-top: 20px;
    }
    th, td {
        border: 1px solid #aaa;
        padding: 8px 10px;
        text-align: center;
    }
    th {
        background-color: #f8f8f8;
        font-weight: bold;
    }
    td.name-cell {
        text-align: left;
    }    
    a {
        color: #007bff;
        text-decoration: none;
    }
    a:hover {
        text-decoration: underline;
    }
    .low-calls {
        font-weight: bold;
        color: #ff0000;
    }
    .low-calls a {
        color: #ff0000;
    }
</style>
</head>
<body>
    <h2>📞 Calling Dashboard - Malad Utsav 2025</h2>
"""

# Summary box HTML (with values properly formatted)
summary_box = f"""    
    <div class="summary-box">
        <p>Total Assigned Calls: <span class="highlight">{total_assigned}</span></p>
        <p>Total Calls Made: <span class="highlight">{total_called}</span></p>
        <p>Progress: <span class="highlight">{call_percentage:.1f}%</span></p>
    </div>
"""

# Table start HTML
table_start = """
    <table>
        <tr>
            <th>Samiti Member</th>
            <th>Total Assigned</th>
            <th>Called</th>
            <th>Confirmed Presence</th>
            <th>Registered 🟢</th>
        </tr>
"""

# Combine the HTML parts
html = html_start + summary_box + table_start

# === ADD ROWS ===
for _, row in summary.iterrows():
    name = row["Samit Member Name"]
    phone = str(row["Samiti Member Phone Number"]).strip()

    # Links
    name_link = f"CallingList{name.replace(' ', '')}.html"
    name_link_full = f"https://maladkshetrasamiti-sketch.github.io/maladutsav2025/CallingList{name.replace(' ', '')}.html"
    tel_link = f"<a href='tel:+91{phone}'>📞</a>"

    # WhatsApp message (plain text — HTML tags won't work inside WhatsApp)
    message = f"Jai Mahesh! Please complete your calling list ASAP.\n\n{name_link_full}\n\n If completed, thumbs up please.🙏"

    # URL encode the message
    encoded_message = urllib.parse.quote_plus(message)

    # WhatsApp link
    whatsapp_link = f"<a href='https://wa.me/91{phone}?text={encoded_message}'>💬</a>"

    # Final combined link display
    name_with_links = f"<small>{tel_link}&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{whatsapp_link}&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href='{name_link}'>{name}</a></small>"

    # Add low-calls class if Called_Count < 10 to make text bold and red
    row_class = ' class="low-calls"' if row['Called_Count'] < 10 else ''

    html += f"""
        <tr>
            <td class='name-cell{" low-calls" if row["Called_Count"] < 10 else ""}'>{name_with_links}</td>
            <td{row_class}>{row['Total_Assigned']}</td>
            <td{row_class}>{row['Called_Count']}</td>
            <td{row_class}>{row['Confirmed_Presence']}</td>
            <td{row_class}>{row['Registered']}</td>
        </tr>
    """

html += """
    </table>
</body>
</html>
"""

# === SAVE OUTPUT ===
with open(output_html, "w", encoding="utf-8") as f:
    f.write(html)

print(f"✅ Dashboard created successfully: {output_html}")

✅ Dashboard created successfully: DashboardSummary.html


In [9]:
import pandas as pd
import os
import urllib.parse
import re

# === CONFIG ===
input_csv = "Calling_List_Karkartas.csv"   # CSV file with data
output_folder = "html_output_karykarta"    # Folder for HTML output
github_base_url = "https://maladkshetrasamiti-sketch.github.io/maladutsav2025"  # Replace with your actual GitHub Pages URL
form_url_base = (
    "https://docs.google.com/forms/d/e/1FAIpQLSddgJb-7qSWjw4vIjB-6rAddcIrCSTreRmbnNNAst2I_bd8dA/viewform?usp=pp_url"
)

# === CREATE OUTPUT FOLDER IF NOT EXISTS ===
os.makedirs(output_folder, exist_ok=True)

# === READ CSV ===
df = pd.read_csv(input_csv)
df.columns = df.columns.str.strip()

# === PROCESS EACH SAMITI MEMBER ===
for samiti_name, group in df.groupby("Samit Member Name"):
    # Clean up filename
    safe_name = samiti_name
    for bad in [" ", ".", ",", "Shri", "Ji", "ji"]:
        safe_name = safe_name.replace(bad, "_")
    safe_name = safe_name.strip("_")

    filename = "MyFile.html"
    filepath = os.path.join(output_folder, filename)

    # GitHub hosted link
    

    html = f"""
    <html>
    <head>
        <meta charset="UTF-8">
        <title>Calling List - {samiti_name}</title>
        <style>
            body {{ font-family: Arial, sans-serif; margin: 20px; }}
            h2 {{ text-align: center; }}
            table {{ width: 100%; border-collapse: collapse; margin-top: 20px; }}
            th, td {{ border: 1px solid #ccc; padding: 8px; text-align: left; }}
            th {{ background-color: #f2f2f2; }}
            a {{ color: #007bff; text-decoration: none; }}
            a:hover {{ text-decoration: underline; }}
        </style>
    </head>
    <body>
        <h2>Calling List - {samiti_name}</h2>
        <table>
            <tr>
                <th>#</th>
                <th>Assigned Person Name</th>
                <th>Mobile Number</th>
                <th>GitHub Link</th>
            </tr>
    """

    group_sorted = group.sort_values(by="Assigned Person")
    i = 1
    for _, row in group_sorted.iterrows():
        assigned_person = row["Assigned Person"]
        safe_assigned_name = assigned_person
        for bad in [" ", ".", ",", "Shri", "Ji", "ji"]:
            safe_assigned_name = safe_assigned_name.replace(bad, "_")

        # Clean the assigned name safely
        safe_assigned_name = (
            assigned_person
            .replace(" ", "")      # Replace spaces with underscores
            .replace("\u00A0", "") # Replace non-breaking spaces
            .replace(".", "")
            .replace(",", "")
        )

        # Remove any extra unwanted characters
        safe_assigned_name = re.sub(r'[^A-Za-z0-9_]', '', safe_assigned_name)
        safe_assigned_name = safe_assigned_name.strip("")

        # Final filename and GitHub link
        filename = f"CallingList{safe_assigned_name}.html"

        # Ensure proper URL encoding
        encoded_filename = urllib.parse.quote(filename, safe="/")
        github_link = f"{github_base_url}/{encoded_filename}"

        mobile = str(row["Mobile Number"]).strip()

        # WhatsApp message with GitHub link
        # WhatsApp message with GitHub link (Motivating + Polite)
        whatsapp_message = (
            "🙏 *Jai Mahesh!*\n"
            f"प्रिय कार्यकर्ता {assigned_person} जी,\n\n"
            "आप *Malad Utsav 2025* 🎉 के इस भव्य आयोजन के अभिन्न अंग हैं। "
            "आपको जिन समाजबंधुओं से संपर्क कर आमंत्रण देना है, उनकी सूची नीचे दिए गए लिंक पर उपलब्ध है 👇\n"
            f"{github_link}\n\n"
            "💡 कृपया निम्नलिखित सरल चरणों का पालन करें:\n"
            "1️⃣ *Mobile Number* पर क्लिक करें — कॉल अपने आप डायल हो जाएगी 📞\n"
            "2️⃣ *SMS* पर क्लिक करें — आपके मोबाइल से आमंत्रण संदेश जाएगा ✉️\n"
            "3️⃣ *WhatsApp* पर क्लिक करें — व्हाट्सएप खुलेगा और संदेश अपने आप पोस्ट हो जाएगा 💬\n"
            "4️⃣ समाजबंधु से बात करने के बाद *Log Call* पर क्लिक करें ताकि आपकी कॉल का विवरण सीधे Google Form पर सेव हो जाए ✅\n\n"
            "📅 *कार्यक्रम तिथि:* 9 नवम्बर 2025 (रविवार)\n"
            "🕠 *समय:* शाम 5:30 बजे से आगे\n"
            "📍 *स्थान:* महेश्वरी प्लॉट, बोरिवली (पश्चिम)\n\n"
            "आपका यह छोटा सा प्रयास हमारे समाज के इस महोत्सव को और भी सफल बनाएगा 🙏\n"
            "कृपया अधिक से अधिक समाजबंधुओं को *Malad Utsav 2025* में आमंत्रित करें और उन्हें परिवार सहित आने के लिए प्रेरित करें 🌸\n\n"
            "सादर धन्यवाद,\n"
            "*Malad Kshetriya Samiti परिवार* 💐"
        )


        encoded_message = urllib.parse.quote_plus(whatsapp_message)
        whatsapp_link = f"https://wa.me/91{mobile}?text={encoded_message}"
        tel_link = f"<a href='tel:+91{mobile}'>+91 {mobile}</a>"

        # # Prefilled Google Form Link
        # call_log_link = (
        #     f"{form_url_base}"
        #     f"&entry.752647600=Called"
        #     f"&entry.1475012068={urllib.parse.quote_plus(assigned_person)}"
        #     f"&entry.1585817478={urllib.parse.quote_plus(mobile)}"
        #     f"&entry.1573933320={urllib.parse.quote_plus(samiti_name)}"
        # )

        html += f"""
            <tr>
                <td>{i}</td>
                <td>{assigned_person}</td>
                <td>{tel_link} | <a onclick=\"this.style.color='green'\" href="{whatsapp_link}" target="_blank">WhatsApp</a></td>
                <td><a href="{github_link}" onclick=\"this.style.color='green'\" target="_blank">GitHub</a></td>
            </tr>
        """
        i += 1

    html += """
        </table>
    </body>
    </html>
    """

    # Save HTML file
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(html)

print(f"✅ HTML files created in folder: {output_folder}")
print(f"✅ Each WhatsApp message now contains its GitHub file link.")


✅ HTML files created in folder: html_output_karykarta
✅ Each WhatsApp message now contains its GitHub file link.
